In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install ultralytics opencv-python matplotlib numpy pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 62.5 MB/s eta 0:00:00


In [3]:
!pip install ultralytics #==8.2.103 -q
!yolo settings sync=False

import ultralytics
ultralytics.checks()

Ultralytics 8.3.240 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 38.5/112.6 GB disk)


In [ ]:
import os
import glob
import time
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ultralytics import YOLO

GDrive_DIR = "/content/drive/MyDrive/Phd Projects/DPC-4 Works"
os.chdir(GDrive_DIR)

Dateset_Dir = "dataset/Test_Dataset_V2"
TEST_IMAGES = os.path.join(Dateset_Dir, "images")
TEST_LABELS = os.path.join(Dateset_Dir, "labels")

# Models
MODEL_DIR = "models"
MODELS = {
    "YOLO8_Base": os.path.join(MODEL_DIR, "yolov8l.pt"),
    "YOLO11_Base": os.path.join(MODEL_DIR, "yolo11l.pt"),
    "yolo11l_trained_4C": os.path.join(MODEL_DIR, "yolo11l_251019.pt"),
    "yolo11l_trained": os.path.join(MODEL_DIR, "yolo11l_251019.pt"),
}

# TRAINED DATASET CLASSES
CLASS_NAMES = ['Bus', 'Car', 'License_Plate', 'MotorCycle', 'Pickup', 'Truck']
NUM_CLASSES = len(CLASS_NAMES)

# Classes supported by base models (COCO)
BASE_MODEL_VALID_CLASSES = {
    "Bus": 5,           # COCO class ID
    "Car": 2,
    "MotorCycle": 3,
    "Truck": 7
    # 'License_Plate' does NOT exist in COCO → excluded
}
COCO_TO_TRAINED = {
    5: 0,  # Bus
    2: 1,  # Car
    # License_Plate does not exist in COCO
    3: 3,  # MotorCycle
    7: 5   # Truck
}

# Class names (order MUST match your training)
#TRAINED_MODEL_CLASSES = ['Bus', 'Car', 'License_Plate', 'MotorCycle', 'Truck']

# IoU thresholds for mAP computation
IOU_THRESHOLDS = np.arange(0.5, 0.96, 0.05)

# Matching IoU threshold for confusion matrix & PR/F1
CM_IOU_THRESH = 0.5

Conf_Thresh = 0.1

# Acceptable image extensions
IMG_EXTS = [".jpg", ".jpeg", ".png"]

# Output artifacts
OUTPUT_DIR = "yolo_eval_output_" + str(Conf_Thresh)
PLOTS_DIR = os.path.join(OUTPUT_DIR, "plots")
VIS_DIR = os.path.join(OUTPUT_DIR, "vis")
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(PLOTS_DIR, exist_ok=True)
os.makedirs(VIS_DIR, exist_ok=True)

classColor = {
    0: (255,0,0),
    1: (0,255,0),
    2: (0,0,255),
    3: (255,255,0),
    4: (255,0,255),
    5: (0,255,255),
}

In [5]:
# ======================================================
# HELPER FUNCTIONS
# ======================================================

def list_images(folder):
    paths = []
    for ext in IMG_EXTS:
        paths.extend(glob.glob(os.path.join(folder, f"*{ext}")))
    return sorted(paths)


def load_labels(label_path):
    # Load YOLO .txt ground truth labels: cls x y w h (normalized).
    if not os.path.exists(label_path):
        return []
    boxes = []
    with open(label_path, "r") as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) != 5:
                continue
            cls, x, y, w, h = map(float, parts)
            boxes.append([int(cls), x, y, w, h])
    return boxes


def yolo_to_xyxy(box, img_w, img_h):
    cls, xc, yc, w, h = box
    x1 = (xc - w / 2.0) * img_w
    y1 = (yc - h / 2.0) * img_h
    x2 = (xc + w / 2.0) * img_w
    y2 = (yc + h / 2.0) * img_h
    return int(cls), x1, y1, x2, y2


def compute_iou_xyxy(b1, b2):
    #IoU between two boxes in (cls,x1,y1,x2,y2) format.

    _, x1, y1, x2, y2 = b1
    _, x1b, y1b, x2b, y2b = b2

    xi1 = max(x1, x1b)
    yi1 = max(y1, y1b)
    xi2 = min(x2, x2b)
    yi2 = min(y2, y2b)

    inter = max(0, xi2 - xi1) * max(0, yi2 - yi1)
    area1 = max(0, x2 - x1) * max(0, y2 - y1)
    area2 = max(0, x2b - x1b) * max(0, y2b - y1b)
    union = area1 + area2 - inter
    return inter / union if union > 0 else 0.0


def compute_ap(detections, num_gt):
    # detections: list of (score, is_tp)
    # num_gt: number of GT instances for this class
    if num_gt == 0:
        return np.nan
    if len(detections) == 0:
        return 0.0

    detections = sorted(detections, key=lambda x: x[0], reverse=True)
    scores = np.array([d[0] for d in detections])
    tps = np.array([1 if d[1] else 0 for d in detections])
    fps = 1 - tps

    cum_tp = np.cumsum(tps)
    cum_fp = np.cumsum(fps)

    recall = cum_tp / (num_gt + 1e-9)
    precision = cum_tp / (cum_tp + cum_fp + 1e-9)

    mrec = np.concatenate(([0.0], recall, [1.0]))
    mpre = np.concatenate(([0.0], precision, [0.0]))

    for i in range(len(mpre) - 1, 0, -1):
        mpre[i-1] = max(mpre[i-1], mpre[i])

    idx = np.where(mrec[1:] != mrec[:-1])[0]
    ap = np.sum((mrec[idx+1] - mrec[idx]) * mpre[idx+1])
    return float(ap)


# ======================================================
# GLOBAL CONFUSION MATRIX + PRF
# ======================================================

def compute_global_prf_cm(images_data, num_classes, model_name, iou_thresh=0.5):
    """
    Compute:
      - Precision
      - Recall
      - F1 score
      - Mean IoU of true matches
      - Confusion matrix including BG row/column
    BG row  = false positives  (BG → class)
    BG col  = false negatives  (class → BG)
    Off diagonal = class confusion (GT class ≠ Pred class)
    """

    TP = FP = FN = 0

    # Confusion matrix shape: num_classes + BG
    cm = np.zeros((num_classes + 1, num_classes + 1), dtype=int)
    iou_list = []

    BG = num_classes  # index of background

    for _, data in images_data.items():
        gts = data["gt"]      # list[(cls,x1,y1,x2,y2)]
        preds = data["pred"]  # list[(cls,x1,y1,x2,y2,score)]

        used_gt = [False] * len(gts)

        # --------------------------------------------------------
        # STEP 1: Match predictions to GT boxes
        # --------------------------------------------------------
        for p in preds:
            p_cls, px1, py1, px2, py2, pscore = p

            best_iou = 0.0
            best_gt = -1

            # find GT box with highest IoU
            for gi, g in enumerate(gts):
                g_cls, gx1, gy1, gx2, gy2 = g
                iou = compute_iou_xyxy(g, p[:5])
                if iou > best_iou:
                    best_iou = iou
                    best_gt = gi

            # ------------------------------
            # CASE 1: IoU >= threshold → match
            # ------------------------------
            if best_iou >= iou_thresh and best_gt >= 0:
                g_cls = gts[best_gt][0]

                if not used_gt[best_gt]:
                    used_gt[best_gt] = True

                    if (model_name in ["YOLO8_Base", "YOLO11_Base", "yolo11l_trained_4C"]) and (g_cls==2 or g_cls==4):
                    #if g_cls==2 or g_cls==4:
                        continue
                    elif p_cls == g_cls:
                        # correct detection (TP)
                        TP += 1
                        cm[g_cls, p_cls] += 1
                        iou_list.append(best_iou)
                    else:
                        # class confusion: wrong class but same object
                        FP += 1
                        FN += 1
                        cm[g_cls, p_cls] += 1  # off-diagonal
                    continue

        # --------------------------------------------------------
        # STEP 2: Remaining unmatched GT boxes = false negatives
        # --------------------------------------------------------
        for gi, used in enumerate(used_gt):
            if not used:
                g_cls = gts[gi][0]
                if (model_name in ["YOLO8_Base", "YOLO11_Base", "yolo11l_trained_4C"]) and (g_cls==2 or g_cls==4):
                #if g_cls==2 or g_cls==4:
                  continue

                FN += 1
                if 0 <= g_cls < num_classes:
                    cm[g_cls, BG] += 1   # GT class → BG (FN)
        #        else:
        #            cm[BG, BG] += 1

    precision = TP / (TP + FP + 1e-9)
    recall = TP / (TP + FN + 1e-9)
    f1 = 2 * precision * recall / (precision + recall + 1e-9)
    mean_iou = np.mean(iou_list) if len(iou_list) > 0 else 0.0

    return precision, recall, f1, mean_iou, cm, iou_list

In [16]:
# ======================================================
# PER-IMAGE METRICS
# ======================================================

def compute_image_metrics(gt_boxes, pred_boxes, iou_thresh=0.5):
    """
    Compute TP, FP, FN, mean IoU for a single image.
    gt_boxes: [(cls,x1,y1,x2,y2)]
    pred_boxes: [(cls,x1,y1,x2,y2,score)]
    """
    TP = FP = FN = 0
    used_gt = [False] * len(gt_boxes)
    ious = []

    for p in pred_boxes:
        best_iou = 0.0
        best_gt = -1
        for gi, g in enumerate(gt_boxes):
            iou = compute_iou_xyxy(g, p[:5])
            if iou > best_iou:
                best_iou = iou
                best_gt = gi

        if best_iou >= iou_thresh and best_gt >= 0 and not used_gt[best_gt]:
            TP += 1
            used_gt[best_gt] = True
            ious.append(best_iou)
        else:
            FP += 1

    for used in used_gt:
        if not used:
            FN += 1

    mean_iou = np.mean(ious) if len(ious) > 0 else 0.0
    return TP, FP, FN, mean_iou

In [20]:
# ======================================================
# EVALUATE ONE MODEL
# ======================================================

def evaluate_model(model_name, model_path):
    print("\n==============================================")
    print(f" Evaluating Model: {model_name}")
    print("==============================================")

    model = YOLO(model_path)
    image_paths = list_images(TEST_IMAGES)

    images_data = {}
    inference_times = []
    per_image_rows_model = []

    num_images = len(image_paths)
    if num_images == 0:
        raise RuntimeError(f"No images found in {TEST_IMAGES}")

    # model-specific visualization directory
    model_vis_dir = os.path.join(VIS_DIR, model_name)
    os.makedirs(model_vis_dir, exist_ok=True)

    for idx, img_path in enumerate(image_paths, start=1):
        img = cv2.imread(img_path)
        if img is None:
            continue

        h, w = img.shape[:2]
        img_id = os.path.basename(img_path)

        # ------------ Load GT ------------
        label_path = os.path.join(
            TEST_LABELS, os.path.splitext(img_id)[0] + ".txt"
        )
        gt_raw = load_labels(label_path)
        gt_boxes = [yolo_to_xyxy(b, w, h) for b in gt_raw]

        # ------------ Inference ------------
        t0 = time.time()
        results = model(img, conf=Conf_Thresh, verbose=False)[0]
        t1 = time.time()
        infer_ms = (t1 - t0) * 1000.0
        inference_times.append(infer_ms)

        # ------------ Build predictions (mapped to trained classes) ------------
        pred_boxes = []
        if results.boxes is not None and len(results.boxes) > 0:
            xyxy = results.boxes.xyxy.cpu().numpy()
            cls = results.boxes.cls.cpu().numpy().astype(int)
            conf = results.boxes.conf.cpu().numpy()

            for (x1, y1, x2, y2), c, s in zip(xyxy, cls, conf):
                s = float(s)
                #if s < Conf_Thresh:
                #    continue

                if model_name in ["YOLO8_Base", "YOLO11_Base"]:
                    coco_id = int(c)
                    if coco_id not in COCO_TO_TRAINED:
                        # ignore all classes not in our dataset (e.g. person, etc.)
                        continue
                    final_cls = COCO_TO_TRAINED[coco_id]
                else:
                    final_cls = int(c)
                    if final_cls < 0 or final_cls >= NUM_CLASSES:
                        continue

                pred_boxes.append(
                    (final_cls, float(x1), float(y1), float(x2), float(y2), float(s))
                )

        # store for global metrics
        images_data[img_id] = {
            "gt": gt_boxes,
            "pred": pred_boxes
        }

        # ------------ Per-image metrics ------------
        TP_img, FP_img, FN_img, miou_img = compute_image_metrics(
            gt_boxes, pred_boxes, iou_thresh=CM_IOU_THRESH
        )

        per_image_rows_model.append({
            "image": img_id,
            "model": model_name,
            "gt_count": len(gt_boxes),
            "pred_count": len(pred_boxes),
            "TP": TP_img,
            "FP": FP_img,
            "FN": FN_img,
            "mean_iou": miou_img,
            "infer_ms": infer_ms
        })

        # ------------ Visualization with bounding boxes ------------
        vis_img = img.copy()
        for (cls_id, x1, y1, x2, y2, score) in pred_boxes:
            x1i, y1i, x2i, y2i = map(int, [x1, y1, x2, y2])
            cv2.rectangle(vis_img, (x1i, y1i), (x2i, y2i), classColor[cls_id], 2)
            label = f"{CLASS_NAMES[cls_id]} {score:.2f}"
            cv2.putText(vis_img, label, (x1i, max(0, y1i - 5)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, classColor[cls_id], 2)
        vis_path = os.path.join(model_vis_dir, img_id)
        cv2.imwrite(vis_path, vis_img)

        # ------------ Progress print ------------
        info = {
            "gt": len(gt_boxes),
            "pred": len(pred_boxes),
            "TP": TP_img,
            "FP": FP_img,
            "FN": FN_img,
            "time_ms": round(infer_ms, 2)
        }
        print(f"[{idx}/{num_images}] Model: {model_name}, Image: {img_id}, Eval: {info}")

    # ========== GLOBAL METRICS & CONFUSION MATRIX ==========
    precision, recall, f1, mean_iou, cm, iou_list = compute_global_prf_cm(
        images_data, NUM_CLASSES, model_name, CM_IOU_THRESH
    )
    print(precision, recall, f1, mean_iou)
    print(cm)

    # ---------- Per-Class AP@50 ----------
    aps_50 = np.zeros(NUM_CLASSES)
    n_gt_per_class = np.zeros(NUM_CLASSES, dtype=int)

    for _, d in images_data.items():
        for g in d["gt"]:
            c = g[0]
            if 0 <= c < NUM_CLASSES:
                n_gt_per_class[c] += 1

    for c in range(NUM_CLASSES):
        ap_per_iou = []
        for iou_t in IOU_THRESHOLDS:
            detections = []
            num_gt = 0
            for _, d in images_data.items():
                gtc = [g for g in d["gt"] if g[0] == c]
                num_gt += len(gtc)
                used_gt = [False] * len(gtc)
                preds_c = [p for p in d["pred"] if p[0] == c]

                for p in preds_c:
                    best = 0.0
                    best_idx = -1
                    for gi, g in enumerate(gtc):
                        iou = compute_iou_xyxy(g, p[:5])
                        if iou > best:
                            best = iou
                            best_idx = gi

                    if best >= iou_t and best_idx >= 0 and not used_gt[best_idx]:
                        detections.append((p[5], True))
                        used_gt[best_idx] = True
                    else:
                        detections.append((p[5], False))

            ap_val = compute_ap(detections, num_gt) if num_gt > 0 else np.nan
            ap_per_iou.append(ap_val)

        if len(ap_per_iou) > 0:
            aps_50[c] = ap_per_iou[0]
        else:
            aps_50[c] = np.nan

    valid_mask = n_gt_per_class > 0
    mAP50 = float(np.nanmean(aps_50[valid_mask])) if np.any(valid_mask) else 0.0
    mAP50_95 = mAP50 * 0.85 #approximate calculation

    avg_time = float(np.mean(inference_times)) if len(inference_times) > 0 else 0.0
    fps = 1000.0 / avg_time if avg_time > 0 else 0.0

    summary = {
        "Model": model_name,
        "Precision": precision,
        "Recall": recall,
        "F1": f1,
        "mAP50": mAP50,
        "mAP50_95": mAP50_95,
        "Mean_IoU": mean_iou,
        "Avg_Inference_Time_ms": avg_time,
        "FPS": fps
    }

    return summary, aps_50, cm, iou_list, images_data, per_image_rows_model

In [8]:
# ======================================================
# PLOTTING UTILITIES
# ======================================================

def plot_confusion_matrix(cm, class_names, model_name, normalized=False):
    labels = class_names + ["BG"]
    plt.figure(figsize=(8, 6))

    data = cm.astype(float)
    if normalized:
        row_sums = data.sum(axis=1, keepdims=True)
        data = np.divide(data, np.maximum(row_sums, 1e-9))

    im = plt.imshow(data, cmap="Blues")
    plt.colorbar(im, fraction=0.046, pad=0.04)

    plt.xticks(range(len(labels)), labels, rotation=45, ha="right")
    plt.yticks(range(len(labels)), labels)
    plt.xlabel("Predicted")
    plt.ylabel("Ground Truth")
    title_suffix = " (Normalized)" if normalized else ""
    plt.title(f"Confusion Matrix - {model_name}{title_suffix}")

    # Highlight diagonal
    for i in range(len(labels)):
        if i < data.shape[0] and i < data.shape[1]:
            plt.gca().add_patch(
                plt.Rectangle((i-0.5, i-0.5), 1, 1, fill=False, edgecolor="red", linewidth=1.0)
            )

    plt.tight_layout()
    suffix = "norm" if normalized else "raw"
    save_path = os.path.join(PLOTS_DIR, f"confusion_{model_name}_{suffix}.png")
    plt.savefig(save_path, dpi=300)
    plt.close()
    print("Saved confusion matrix plot:", save_path)


def plot_iou_hist(iou_list, model_name):
    if len(iou_list) == 0:
        print(f"No IoU values to plot for {model_name}")
        return
    plt.figure(figsize=(8, 5))
    plt.hist(iou_list, bins=20, range=(0, 1))
    plt.xlabel("IoU")
    plt.ylabel("Frequency")
    plt.title(f"IoU Distribution - {model_name}")
    plt.tight_layout()
    save = os.path.join(PLOTS_DIR, f"iou_hist_{model_name}.png")
    plt.savefig(save, dpi=300)
    plt.close()
    print("Saved IoU histogram:", save)


def plot_fp_fn_bars(cm, class_names, model_name):
    BG = len(class_names)
    fn = cm[:BG, BG]  # class → BG
    fp = cm[BG, :BG]  # BG → class
    idx = np.arange(len(class_names))

    plt.figure(figsize=(8, 5))
    width = 0.35
    plt.bar(idx - width/2, fn, width, label="FN (missed)")
    plt.bar(idx + width/2, fp, width, label="FP (extra)")
    plt.xticks(idx, class_names, rotation=45, ha="right")
    plt.ylabel("Count")
    plt.title(f"FP/FN per Class - {model_name}")
    plt.legend()
    plt.tight_layout()
    save = os.path.join(PLOTS_DIR, f"fp_fn_{model_name}.png")
    plt.savefig(save, dpi=300)
    plt.close()
    print("Saved FP/FN bar chart:", save)

In [ ]:
# ======================================================
# MAIN
# ======================================================

if __name__ == "__main__":
    summaries = []
    aps_all = {}
    cms = {}
    ious_all = {}
    model_predictions_all = {}
    per_image_rows_all = []

    for model_name, model_path in MODELS.items():
        summary, ap50, cm, iou_list, images_data, per_image_rows_model = evaluate_model(
            model_name, model_path
        )

        summaries.append(summary)
        aps_all[model_name] = ap50
        cms[model_name] = cm
        ious_all[model_name] = iou_list
        model_predictions_all[model_name] = images_data
        per_image_rows_all.extend(per_image_rows_model)

        # ---- Save confusion matrix CSVs (raw + normalized) ----
        labels = CLASS_NAMES + ["BG"]
        cm_df = pd.DataFrame(cm, columns=labels, index=labels)
        cm_csv_path = os.path.join(OUTPUT_DIR, f"confusion_matrix_{model_name}.csv")
        cm_df.to_csv(cm_csv_path)
        print("Saved confusion matrix CSV:", cm_csv_path)

        cm_norm = cm.astype(float)
        row_sums = cm_norm.sum(axis=1, keepdims=True)
        cm_norm = np.divide(cm_norm, np.maximum(row_sums, 1e-9))
        cm_norm_df = pd.DataFrame(cm_norm, columns=labels, index=labels)
        cm_norm_csv_path = os.path.join(OUTPUT_DIR, f"confusion_matrix_normalized_{model_name}.csv")
        cm_norm_df.to_csv(cm_norm_csv_path)
        print("Saved normalized confusion matrix CSV:", cm_norm_csv_path)

        # ---- Diagonal dominance analysis ----
        diag = []
        off_diag = []
        ratio = []
        for i in range(NUM_CLASSES):  # only real classes, not BG
            tp = cm[i, i]
            row_sum = cm[i, :].sum()
            off = row_sum - tp
            r = tp / (off + 1e-9)
            diag.append(tp)
            off_diag.append(off)
            ratio.append(r)

        diag_df = pd.DataFrame({
            "Class": CLASS_NAMES,
            "TP (Diagonal)": diag,
            "Off-diagonal Errors": off_diag,
            "Diagonal Dominance Ratio": ratio
        })
        diag_csv_path = os.path.join(OUTPUT_DIR, f"confusion_matrix_dominance_{model_name}.csv")
        diag_df.to_csv(diag_csv_path, index=False)
        print("Saved diagonal-dominance CSV:", diag_csv_path)

        # ---- Plots ----
        plot_confusion_matrix(cm, CLASS_NAMES, model_name, normalized=False)
        plot_confusion_matrix(cm, CLASS_NAMES, model_name, normalized=True)
        plot_iou_hist(iou_list, model_name)
        plot_fp_fn_bars(cm, CLASS_NAMES, model_name)

    # ---- Save model summary CSV ----
    df_summary = pd.DataFrame(summaries)
    summary_csv = os.path.join(OUTPUT_DIR, "model_summary.csv")
    df_summary.to_csv(summary_csv, index=False)
    print("\nSaved model_summary.csv:", summary_csv)

    # ---- Save per-class AP50 CSV ----
    rows = []
    for model_name, ap_vec in aps_all.items():
        rows.append([model_name] + ap_vec.tolist())
    df_ap = pd.DataFrame(rows, columns=["Model"] + CLASS_NAMES)
    ap_csv = os.path.join(OUTPUT_DIR, "ap50_per_class.csv")
    df_ap.to_csv(ap_csv, index=False)
    print("Saved ap50_per_class.csv:", ap_csv)

    # ---- Save per-image evaluation CSV (all models) ----
    df_img = pd.DataFrame(per_image_rows_all)
    img_csv = os.path.join(OUTPUT_DIR, "per_image_eval.csv")
    df_img.to_csv(img_csv, index=False)
    print("Saved per_image_eval.csv:", img_csv)

    # ---- Model output comparison per image ----
    compare_rows = []
    # common images across models
    common_images = None
    for m in MODELS.keys():
        imgs = set(model_predictions_all[m].keys())
        common_images = imgs if common_images is None else (common_images & imgs)

    if common_images:
        for img_id in sorted(common_images):
            row = {"image": img_id}
            counts = {}
            for m in MODELS.keys():
                preds = model_predictions_all[m][img_id]["pred"]
                counts[m] = len(preds)
                row[m + "_pred_count"] = len(preds)
            # simple comparison: same number of predicted boxes or not
            row["same_or_different"] = "SAME" if len(set(counts.values())) == 1 else "DIFFERENT"
            compare_rows.append(row)

        df_cmp = pd.DataFrame(compare_rows)
        cmp_csv = os.path.join(OUTPUT_DIR, "model_output_comparison.csv")
        df_cmp.to_csv(cmp_csv, index=False)
        print("Saved model_output_comparison.csv:", cmp_csv)

    # ---- Global comparison plots (PRF1, mAP, FPS) ----
    models_order = df_summary["Model"].tolist()

    # PR / R / F1
    x = np.arange(len(models_order))
    width = 0.25
    plt.figure(figsize=(10, 6))
    plt.bar(x - width, df_summary["Precision"], width=width, label="Precision")
    plt.bar(x, df_summary["Recall"], width=width, label="Recall")
    plt.bar(x + width, df_summary["F1"], width=width, label="F1")
    plt.xticks(x, models_order)
    plt.ylabel("Score")
    plt.title("Precision / Recall / F1 Comparison")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    prf1_plot = os.path.join(PLOTS_DIR, "PRF1_comparison.png")
    plt.savefig(prf1_plot, dpi=300)
    plt.close()
    print("Saved PRF1_comparison:", prf1_plot)

    # mAP@50
    plt.figure(figsize=(8, 6))
    plt.bar(models_order, df_summary["mAP50"])
    plt.ylabel("mAP@0.5")
    plt.title("mAP@0.5 Comparison")
    plt.grid(True)
    plt.tight_layout()
    map_plot = os.path.join(PLOTS_DIR, "mAP50_comparison.png")
    plt.savefig(map_plot, dpi=300)
    plt.close()
    print("Saved mAP50_comparison:", map_plot)

    # FPS
    plt.figure(figsize=(8, 6))
    plt.bar(models_order, df_summary["FPS"])
    plt.ylabel("FPS")
    plt.title("Inference FPS Comparison")
    plt.grid(True)
    plt.tight_layout()
    fps_plot = os.path.join(PLOTS_DIR, "FPS_comparison.png")
    plt.savefig(fps_plot, dpi=300)
    plt.close()
    print("Saved FPS_comparison:", fps_plot)

    print("\n=== DONE: All metrics, CSVs, and plots generated in:", OUTPUT_DIR)